# original.txtの中身を調べる

* すでに表記ゆれはある程度前処理されている
* original.txtからデータセットを作成する方法はいくつかある
* capoの方は、半音下げ(-1) ~ capo5(-5)までが大半を占めているので、6と7は削除でよさそう
* chordの方はいろいろある。低頻度のコードを削除する、何らかのコードで置き換えるなど
* とりあえず削除する方向で

In [2]:
from pathlib import Path

In [3]:
orignal_path=Path("../data/original.txt")

In [4]:
def get_song_list(original_path):
    with orignal_path.open("r") as f:
        songs = f.readlines()
    result=[]
    for song in songs:
        song=song.rstrip('\n')
        divided=song.split(" ")
        song_dic={"chords":divided[:-1],"rec_capo":divided[-1]}
        result.append(song_dic)
    return result

def remove_rare_capo_song(songs):
    """-6,-7はほとんどないので除去する."""
    removed=[]
    for song in songs:
        if song["rec_capo"] not in ['-6','-7']:
            removed.append(song)
    return removed


from sklearn.model_selection import train_test_split

def split_dataset(songs,val_rate, test_rate,seed=None):
    songs_train_val,songs_test= train_test_split(songs,test_size=test_rate,random_state=seed)
    songs_train,songs_val= train_test_split(songs,test_size=val_rate,random_state=seed)
    return songs_train,songs_val,songs_test

In [5]:
from collections import Counter

def retrieve_chord_stat(songs):
    chord_dict = Counter()
    for song in songs:
        for chord in song["chords"]: 
            chord_dict[chord]+=1
    return chord_dict

def retrieve_capo_stat(songs):
    capo_dict =  Counter()
    for song in songs:
        capo_dict[song["rec_capo"]]+=1
    return capo_dict

In [6]:
songs=get_song_list(orignal_path)
songs=remove_rare_capo_song(songs)
songs_train, songs_valid, songs_test=split_dataset(songs,0.1,0.2)

In [7]:
chord_stat=retrieve_chord_stat(songs_train)
capo_stat=retrieve_capo_stat(songs_train)

## chordの出現頻度調査
* 低頻度chordはparseミスとかも含むので除去したい
* 5回以下しか出現しない約140個のコードを対象？

In [8]:
import numpy as np
import matplotlib.pyplot as plt

print(f"Total chords :{len(chord_stat.keys())}")

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
edges = range(0,100,5)
ax.hist(list(chord_stat.values()), bins=edges)
ax.set_title('histogram of chord')
ax.set_xlabel('frequency')
ax.set_ylabel('the number of chords')
fig.show()

Total chords :847


/Users/hikaru_ogura/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/matplotlib/figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


In [49]:
def tokenize_rare_chord(chords_list,onehot_dict,token):
    result=[]
    for chords in chords_list:
        result_inner=[]
        for i,chord in enumerate(chords):
            if chord not in onehot_dict.keys():
                result_inner.append(token)
            else:
                result_inner.append(chord)
        result.append(result_inner)
    return result

def get_onehot_dict(chord_stat,threshold,token):
    onehot_dict={}
    i=0
    for chord,freq in chord_stat.items():
        if freq>threshold:
            onehot_dict[chord]=i
            i+=1
    onehot_dict[token]=i
    return onehot_dict  

def count_and_normalize_chord_features(tokenized,onehot_dict):
    X=[]
    features_num=len(onehot_dict)
    for chords in tokenized:
        x=[0]*features_num
        for chord in chords:
            x[onehot_dict[chord]]+=1
        x=np.array(x)
        X.append(x/sum(x))
    return np.array(X)

def get_chord_features(songs,onehot_dict=None,threshold=5,token="<UNK>"):
    """train : onehot_dictがNone. valid or test : onehot_dictを与える."""
    chords_list=[song["chords"] for song in songs]
    if onehot_dict is None:
        chord_stat=retrieve_chord_stat(songs)
        onehot_dict=get_onehot_dict(chord_stat,threshold,token)
    tokenized=tokenize_rare_chord(chords_list,onehot_dict,token)
    X=count_and_normalize_chord_features(tokenized,onehot_dict)
    return X, onehot_dict


In [50]:
chord_features_train, onehot_dict=get_chord_features(songs_train)
chord_features_valid, _ =get_chord_features(songs_valid,onehot_dict)
chord_features_test, _ =get_chord_features(songs_test,onehot_dict)

In [51]:
chord_features_train.shape,chord_features_valid.shape,chord_features_test.shape

((35995, 684), (4000, 684), (7999, 684))

In [72]:
expanded_chord_features_train=chord_features_train.repeat(7,axis=0)
expanded_chord_features_valid = chord_features_valid.repeat(7,axis=0)
expanded_chord_features_test = chord_features_test.repeat(7,axis=0)

In [157]:
def duplicate_capos(n,dupulication_capo_order=np.array(['1','0','-1','-2','-3','-4','-5'])):
    """n回dupulication_capo_orderを繰り返す."""
    return np.tile(dupulication_capo_order,n)

from sklearn.preprocessing import LabelBinarizer

def capo_onehot_encode(capo):
    """文字列のcapoをonehot encoding"""
    binarizer=LabelBinarizer()
    binarized=binarizer.fit_transform(capo)
    return binarized, binarizer

def create_compares(songs,dupulication_capo_order=np.array(['1','0','-1','-2','-3','-4','-5'])):
    compares=[]
    for i, song in enumerate(songs_train):
        rec=song["rec_capo"]
        pos_j=int(np.where(dupulication_capo_order==rec)[0])
        for j in range(7):
            if j != pos_j:
                compares.append([7*i + pos_j, 7*i + j])
    return np.array(compares)

In [158]:
compares=create_compares(songs_train)

In [159]:
duplicated_capos_train=duplicate_capos(len(songs_train))
duplicated_capos_valid = duplicate_capos(len(songs_valid))
duplicated_capos_test = duplicate_capos(len(songs_test))
capo_features_train,capo_encoder = capo_onehot_encode(duplicated_capos_train)
capo_features_valid = capo_encoder.transform(duplicated_capos_valid)
capo_features_test = capo_encoder.transform(duplicated_capos_test)

In [160]:
X_train=np.concatenate([expanded_chord_features_train, capo_features_train], 1)
X_valid = np.concatenate([expanded_chord_features_valid, capo_features_valid], 1)
X_test = np.concatenate([expanded_chord_features_test, capo_features_test], 1)

In [161]:
X_train.shape, X_valid.shape, X_test.shape

((251965, 691), (28000, 691), (55993, 691))

In [162]:
from scipy import sparse
X_train_sp=sparse.csr_matrix(X_train)
X_valid_sp=sparse.csr_matrix(X_valid)
X_test_sp=sparse.csr_matrix(X_test)

In [287]:
def predict_ranking(X,model):
    pred=model.predict(X)
    reshaped=np.reshape(pred,(-1,7))
    results=[]
    for i in range(len(reshaped)):
        results.append(return_sorted_capo(reshaped[i]))
    return results

def predict_top1(X,model):
    rankings=predict_ranking(X,model)
    return np.array([ranking[0] for ranking in rankings]) 

from sklearn.metrics import accuracy_score

def evaluate_top1(X,model,y):
    pred=predict_top1(X,model)
    return accuracy_score(y, pred)

def return_sorted_capo(pred,dupulication_capo_order=np.array(['1','0','-1','-2','-3','-4','-5'])):
    index = [0]*7
    for j, rank in enumerate(np.argsort(pred[::-1])):
        index[rank] = j
    return dupulication_capo_order[index] 

In [300]:
from fastFM import bpr
model=bpr.FMRecommender(n_iter=5)
model.fit(X_train_sp,compares)

FMRecommender(init_stdev=0.1, l2_reg=None, l2_reg_V=0.1, l2_reg_w=0.1,
       n_iter=5, random_state=123, rank=8, step_size=0.1)

In [301]:
rec_capo_valid= np.array([song["rec_capo"] for song in songs_valid])
ranking=predict_ranking(X_valid_sp,model)
top1= predict_top1(X_valid_sp,model)
evaluate_top1(X_valid_sp,model,rec_capo_valid)

0.12275

In [302]:
ranking[:10]

[array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-2', '1', '-4', '-5', '-3'], dtype='<U2'),
 array(['-1', '0', '-2', '1', '-4', '-5', '-3'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2'),
 array(['-1', '0', '-3', '1', '-4', '-5', '-2'], dtype='<U2')]

In [285]:
embeddings=model.V_.T

In [211]:
capo_embeddings = embeddings[-7:]

In [212]:
capo_embeddings

array([[ 5.72525806e-03, -4.63672084e-03, -6.88011226e-03,
        -1.75563180e-03, -1.25372914e-03, -4.55639374e-04,
        -3.29538733e-03,  5.54391736e-04,  3.53605203e-03,
        -2.42420640e-04,  1.98761894e-03,  6.44847856e-03,
        -1.98238362e-03, -1.60657734e-03,  2.69696275e-03,
         2.36151007e-03],
       [-7.32288343e-04, -1.55619940e-03, -2.95632501e-03,
        -4.84173474e-03,  2.27155596e-03,  1.42498222e-03,
        -1.10259453e-03,  9.05069140e-03, -3.04951741e-03,
         3.52328179e-03, -1.98937473e-03,  4.07732600e-03,
        -2.46283984e-03,  1.31779657e-03, -2.71511166e-03,
         5.70236407e-03],
       [ 4.47649548e-04,  2.38963465e-03, -2.00120426e-03,
         4.29996936e-03, -3.98560498e-04, -1.46546104e-03,
        -1.67172774e-05, -3.61926042e-03,  2.93326806e-03,
        -8.49353556e-03, -4.12336400e-03,  3.41224637e-04,
        -1.64173146e-03,  1.72502655e-03, -1.25412440e-03,
        -2.84437442e-03],
       [-1.61690701e-03,  8.90677568e